In [2]:
!pip install ultralytics

In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO("/content/runs/detect/train6/weights/last.pt")  # load a pretrained model (recommended for training)


In [ ]:
import os
import shutil

# Đường dẫn đến thư mục nguồn (source)
source_folder = "/content/drive/MyDrive/heineken_img"

# Đường dẫn đến thư mục đích (destination)
destination_folder = "/content/image"

# Lấy danh sách các tệp trong thư mục nguồn
files = os.listdir(source_folder)

# Tạo thư mục đích nếu chưa tồn tại
os.makedirs(destination_folder, exist_ok=True)

# Di chuyển từng tệp từ thư mục nguồn sang thư mục đích
for file in files:
    source_file = os.path.join(source_folder, file)
    destination_file = os.path.join(destination_folder, file)
    shutil.copy(source_file, destination_file)
    print(f"Đã sao chép '{file}' thành công.")

print("Đã sao chép toàn bộ tệp từ thư mục nguồn sang thư mục đích.")


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Đường dẫn tới các thư mục chứa hình ảnh và nhãn
image_dir = '/content/image'
label_dir = '/content/drive/MyDrive/labels'

# Tạo các thư mục mới cho training, validation và test
train_image_dir = '/content/train/images'
train_label_dir = '/content/train/labels'
val_image_dir = '/content/val/images'
val_label_dir = '/content/val/labels'
test_dir = '/content/test/images'

for dir_path in [train_image_dir, train_label_dir, val_image_dir, val_label_dir, test_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Đọc danh sách các file hình ảnh và nhãn
images = os.listdir(image_dir)
labels = os.listdir(label_dir)

# Tạo từ điển để kiểm tra nhanh nhãn có tồn tại cho một hình ảnh
label_dict = {label.split('.')[0]: label for label in labels}

# Phân loại hình ảnh có nhãn và không nhãn
labeled_images = []
unlabeled_images = []

for image in images:
    if image.split('.')[0] in label_dict:
        labeled_images.append(image)
    else:
        unlabeled_images.append(image)

# Chia tập có nhãn thành training và validation (mặc định 80/20)
train_images, val_images = train_test_split(labeled_images, test_size=0.20, random_state=42)

# Di chuyển các file vào thư mục tương ứng
def move_files(file_list, source_dir, target_dir):
    for file in file_list:
        shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, file))

# Di chuyển hình ảnh và nhãn vào thư mục train và val
move_files(train_images, image_dir, train_image_dir)
move_files([label_dict[img.split('.')[0]] for img in train_images], label_dir, train_label_dir)
move_files(val_images, image_dir, val_image_dir)
move_files([label_dict[img.split('.')[0]] for img in val_images], label_dir, val_label_dir)

# Di chuyển hình ảnh không có nhãn vào thư mục test
move_files(unlabeled_images, image_dir, test_dir)


In [ ]:
len(os.listdir('/content/data/train/images'))

947

In [19]:
import yaml

# Thông tin cấu hình mô hình YOLO
config_data = {
    'train': '/content/data/train/images',   # Thay đổi đường dẫn này
    'val': '/content/data/val',       # Thay đổi đường dẫn này      # Thay đổi đường dẫn này
    'nc': 11,
    'names': [
        'Billboard Tiger', 'Billboard Heineken', 'Poster Tiger', 'Poster Heineken',
        'Banner Tiger', 'Banner Heineken', 'Beer Bottle', 'Tiger Product',
        'Heineken Product', 'Beer carton Tiger', 'Beer carton Heineken']
}

In [20]:
with open('yolo_config.yaml', 'w') as file:
    yaml.dump(config_data, file, sort_keys=False)

In [6]:
results = model.train(data="/content/yolo_config.yaml", epochs=100, imgsz=640)

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train6/weights/last.pt, data=/content/yolo_config.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

train: Scanning /content/data/train/labels.cache... 947 images, 471 backgrounds, 2 corrupt: 100%|██████████| 947/947 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/data/train/images/387756952_1490853628418507_2595351551210020553_n.jpg: ignoring corrupt image/label: negative label values [  -0.016618   -0.019688   -0.022139   -0.000542]
train: WARNING ⚠️ /content/data/train/images/387799422_1566203864210816_4555076786537359468_n.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2791]
train: WARNING ⚠️ /content/data/train/images/66507344_1706869582276.jpg: 2 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/data/val/labels.cache... 237 images, 123 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train7
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100       2.8G     0.6617      0.372     0.8245          0        640: 100%|██████████| 60/60 [00:06<00:00,  9.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:00<00:00,  8.41it/s]


                   all        237       1439      0.681      0.278      0.336      0.262

100 epochs completed in 0.233 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 6.2MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n summary (fused): 168 layers, 3007793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05<00:00,  1.44it/s]


                   all        237       1439      0.524      0.333      0.358      0.291
       Billboard Tiger          4          5      0.861        0.2        0.2       0.18
          Poster Tiger          9         16      0.555       0.25      0.302      0.227
       Poster Heineken          1          1      0.673          1      0.995      0.995
          Banner Tiger          4         10          1          0      0.182      0.102
           Beer Bottle          6         21          0          0     0.0174    0.00457
         Tiger Product          9        158     0.0618    0.00633     0.0121    0.00626
      Heineken Product          2         15          0          0          0          0
     Beer carton Tiger        103        828      0.795      0.688       0.69        0.5
  Beer carton Heineken         90        385      0.774      0.849      0.826      0.606
Speed: 0.3ms preprocess, 11.1ms inference, 0.0ms loss, 4.9ms postprocess per image
Results saved to runs/detec

In [13]:
model.predict("/content/71ffd7ce-11ad-44e4-9cea-cca37b281951.jpg", save=True, save_txt=True, imgsz=640)


image 1/1 /content/71ffd7ce-11ad-44e4-9cea-cca37b281951.jpg: 448x640 3 Beer carton Tigers, 2 Beer carton Heinekens, 10.1ms
Speed: 3.3ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/train75
1 label saved to runs/detect/train75/labels


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Billboard Tiger', 1: 'Billboard Heineken', 2: 'Poster Tiger', 3: 'Poster Heineken', 4: 'Banner Tiger', 5: 'Banner Heineken', 6: 'Beer Bottle', 7: 'Tiger Product', 8: 'Heineken Product', 9: 'Beer carton Tiger', 10: 'Beer carton Heineken'}
 obb: None
 orig_img: array([[[156, 163, 172],
         [154, 161, 170],
         [151, 158, 167],
         ...,
         [ 71,  77,  84],
         [ 69,  75,  82],
         [ 67,  73,  80]],
 
        [[ 45,  52,  61],
         [ 44,  51,  60],
         [ 42,  49,  58],
         ...,
         [ 71,  77,  84],
         [ 69,  75,  82],
         [ 67,  73,  80]],
 
        [[ 50,  57,  66],
         [ 49,  56,  65],
         [ 48,  55,  64],
         ...,
         [ 69,  77,  84],
         [ 67,  75,  82],
         [ 65,  73,  80]],
 
        ...,
 
        [[105,  81,  69],
         [103,  75,  64],
  